# Tutorial on metaclasses

In [2]:
class A(object):
    pass

type(A)

type

In [1]:
class A: # compiled at python2 to show python2 effect.
    pass

type(A)

classobj

In [20]:
class A:
    __metaclass__ = type
    pass

type(A)

type

In [9]:
# it returns the number of base classes for the particular class
A.__bases__ 

(object,)

In [1]:
class B(metaclass=type):
    pass

In [1]:
class C(object):
    attr = 10
    pass


In [2]:
C = type('C', (object, ), {'attr' : 10})

In [3]:
C

__main__.C

So you did notice something interesting. We can create classes dynamically. So it happens at times we dont have the knowledge of the methods the class will contain the name of the class. So this is a best option. 

In [10]:
def __init__(self, value):
    self.value = value
    pass

def add(self, a, b):
    return a + b

In [11]:
C = type('C', (object, ), {'attr' : 10,
                           '__init__' : __init__,
                           'add' : add})

In [12]:
c = C(10)

In [13]:
c.add(2, 3)

5

In [2]:
class MyMeta(type):
    def __str__(cls):
        return 'hello'

In [17]:
#python3 syntax

class A(metaclass=MyMeta):
    pass

In [18]:
str(A)

'hello'

In [19]:
str(C)

"<class '__main__.C'>"

Try to understand what just happened, I used the instance a meta class I created, and override the descriptor str and thus it prints "hello".

In [3]:
#python2 syntax

class A:
    __metaclass__ = MyMeta
    pass

In [4]:
str(A)

'hello'

In [1]:
class MyMeta(type):
    
    def __new__(mcl, name ,bases, cdict):
        print (mcl)
        print (name)
        print (bases)
        print (cdict) # class dictionary with all the methods inside
        return super(MyMeta, mcl).__new__(mcl, name ,bases, cdict)

#### tutorial on new

* ```__new__``` is called when object is created
* ```__init__``` is called when object is initialised.

So new is called before init

You can notice New is a method is in actual invoked.

In [5]:
class New(object, metaclass = MyMeta):
    pass


<class '__main__.MyMeta'>
New
(<class 'object'>,)
{'__qualname__': 'New', '__module__': '__main__'}


In [9]:
class MyMeta(type):
    
    def __init__(cls, name, bases, cdict):
        print (cls)
        print (name)
        print (bases)
        print (cdict)
        super(MyMeta, cls).__init__(name, bases, cdict)
        pass
    pass


In [10]:
class New(object, metaclass = MyMeta):
    def meth():
        pass
    
    pass


<class '__main__.New'>
New
(<class 'object'>,)
{'__qualname__': 'New', '__module__': '__main__', 'meth': <function New.meth at 0x7ff8f40582f0>}


In [11]:
class B(object):
    
    def __new__(cls, value):
        super().__new__(cls)
        pass
    
    def __init__(self, value):
        self.value = value
        pass
    
    pass


In [12]:
b = B(10)

In [ ]:
# %load deco_meta/metaclasses/meta_2_3.py
# file: meta_2_3.py

"""
The code is a bit hard to understand. The basic idea is exploiting the idea
that metaclasses can customize class creation and are picked by by the parent
class. This particular implementation uses a metaclass to remove its own parent
from the inheritance tree on subclassing. The end result is that the function
creates a dummy class with a dummy metaclass. Once subclassed the dummy
classes metaclass is used which has a constructor that basically instances a
new class from the original parent and the actually intended metaclass.
That way the dummy class and dummy metaclass never show up.

From:
 http://lucumr.pocoo.org/2013/5/21/porting-to-python-3-redux/#metaclass-syntax-changes

Used in:

* Jinja2
* SQLAlchemy
* future (python-future.org)

"""

from __future__ import print_function
import platform


# from jinja2/_compat.py
def with_metaclass(meta, *bases):
    # This requires a bit of explanation: the basic idea is to make a
    # dummy metaclass for one level of class instanciation that replaces
    # itself with the actual metaclass.  Because of internal type checks
    # we also need to make sure that we downgrade the custom metaclass
    # for one level to something closer to type (that's why __call__ and
    # __init__ comes back from type etc.).
    #
    # This has the advantage over six.with_metaclass in that it does not
    # introduce dummy classes into the final MRO.
    class metaclass(meta):
        __call__ = type.__call__
        __init__ = type.__init__

        def __new__(cls, name, this_bases, d):
            if this_bases is None:
                return type.__new__(cls, name, (), d)
            return meta(name, bases, d)

    return metaclass('temporary_class', None, {})


if __name__ == '__main__':

    class BaseClass(object):
        pass


    class MetaClass(type):
        """Metaclass for Python 2 and 3.
        """
        def __init__(cls, name, bases, cdict):
            print('It works with {impl} version {ver}.'.format(
                impl=platform.python_implementation(),
                ver=platform.python_version()))
            super(MetaClass, cls).__init__(name, bases, cdict)


    class Class(with_metaclass(MetaClass, BaseClass)):
        # BaseClass is optional.
        pass
